In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from torchinfo import summary

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
class BostonDataset(torch.utils.data.Dataset):
    '''
    Prepare the Boston dataset for regression
    '''
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            # Apply scaling if necessary
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [4]:
class MLP(nn.Module):
    '''
    Multilayer Perceptron for regression.
    '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
          nn.Linear(13, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)
        )


    def forward(self, x):
        '''
          Forward pass
        '''
        return self.layers(x)

In [5]:
if __name__ == '__main__':
    # Set fixed random number seed
    torch.manual_seed(42)
  
    # Load Boston dataset
    X, y = load_boston(return_X_y=True)

/home/isidro/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing data

In [26]:
len(X), type(X), np.shape(X)

(506, numpy.ndarray, (506, 13))

In [6]:
# Prepare Boston dataset
dataset = BostonDataset(X, y)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [25]:
dataset.X

tensor([[-0.4198,  0.2848, -1.2879,  ..., -1.4590,  0.4411, -1.0756],
        [-0.4173, -0.4877, -0.5934,  ..., -0.3031,  0.4411, -0.4924],
        [-0.4173, -0.4877, -0.5934,  ..., -0.3031,  0.3964, -1.2087],
        ...,
        [-0.4134, -0.4877,  0.1157,  ...,  1.1765,  0.4411, -0.9830],
        [-0.4078, -0.4877,  0.1157,  ...,  1.1765,  0.4032, -0.8653],
        [-0.4150, -0.4877,  0.1157,  ...,  1.1765,  0.4411, -0.6691]],
       dtype=torch.float64)

In [36]:
# Initialize the MLP
mlp = MLP()
mlp.float()
# Define the loss function and optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [37]:
# Run the training loop
for epoch in range(0, 5): # 5 epochs at maximum  
    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

          # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

Starting epoch 1
Loss after mini-batch     1: 0.053
Loss after mini-batch    11: 0.478
Loss after mini-batch    21: 0.407
Loss after mini-batch    31: 0.432
Loss after mini-batch    41: 0.465
Loss after mini-batch    51: 0.478
Starting epoch 2
Loss after mini-batch     1: 0.033
Loss after mini-batch    11: 0.442
Loss after mini-batch    21: 0.443
Loss after mini-batch    31: 0.461
Loss after mini-batch    41: 0.448
Loss after mini-batch    51: 0.473
Starting epoch 3
Loss after mini-batch     1: 0.041
Loss after mini-batch    11: 0.451
Loss after mini-batch    21: 0.446
Loss after mini-batch    31: 0.430
Loss after mini-batch    41: 0.460
Loss after mini-batch    51: 0.451
Starting epoch 4
Loss after mini-batch     1: 0.037
Loss after mini-batch    11: 0.419
Loss after mini-batch    21: 0.455
Loss after mini-batch    31: 0.462
Loss after mini-batch    41: 0.446
Loss after mini-batch    51: 0.444
Starting epoch 5
Loss after mini-batch     1: 0.032
Loss after mini-batch    11: 0.482
Loss 

In [38]:
summary(mlp, batch_size=-1)

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       896
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       2,080
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       33
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0

In [47]:
mlp.eval()

MLP(
  (layers): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [53]:
# test = np.random.randn(13)
# torch usa tensores de torch y no numpy.darrays
dtype = torch.float
test = torch.randn((1, 13), device=device, dtype=dtype, )

In [54]:
mlp.forward(test)

tensor([[0.7430]], grad_fn=<AddmmBackward0>)